## Preliminaries

### Paths

In [1]:
import os
import pathlib
import sys

In [2]:
if not 'google.colab' in str(get_ipython()):
    
    notebooks = os.getcwd()
    parent = str(pathlib.Path(notebooks).parent)
    sys.path.append(parent)

<br>

Warehouse

```python
warehouse = os.path.join(parent, 'warehouse')
if not os.path.exists(warehouse):
    os.makedirs(warehouse)
```

<br>
<br>

### Libraries

In [3]:
%matplotlib inline

import logging
import collections

import numpy as np
import pandas as pd
import xlrd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


<br>

### Logging

In [4]:
logging.basicConfig(level=logging.INFO,
                    format='\n%(message)s\n%(asctime)s.%(msecs)03d\n',
                    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)

<br>

## Data

<br>

Exploring via all expected age groups and the year 2011

In [5]:
age_groups = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44',
              '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90+']

In [6]:
year = '2011'

<br>
<br>

### Patients

<br>

The number of patients **(a)** that originated from each MSOA during the 2011, **(b)** that each NHS Trust received from one or more MSOA entities.

In [7]:
uri = '../warehouse/patients/{year}.csv'.format(year=year)

try:
    patients = pd.read_csv(filepath_or_buffer=uri, header=0, encoding='utf-8')
except RuntimeError as err:
    raise Exception(err)
    
logger.info(patients.head())


   catchment_year       msoa trust_code  patients_from_msoa_to_trust  \
0            2011  E02001214        R0A                         1981   
1            2011  E02001240        R0A                         1181   
2            2011  E02001057        R0A                         1752   
3            2011  E02006902        R0A                          906   
4            2011  E02001056        R0A                         2002   

   total_patients_of_msoa  
0                    3418  
1                    2944  
2                    3421  
3                    1791  
4                    4350  
2021-12-19 21:50:15.307



<br>
<br>

The number of NHS Trusts

In [8]:
patients.trust_code.unique().shape[0]

139

<br>
<br>
<br>

### Populations

<br>

The 2011 population data

In [9]:
uri = '../warehouse/populations/msoa/group/{year}.csv'.format(year=year)

try:
    populations = pd.read_csv(filepath_or_buffer=uri, header=0, encoding='utf-8')
except RuntimeError as err:
    raise Exception(err)

In [10]:
populations.head()

,msoa,ltla,sex,0-4,5-9,10-14,15-19,20-24,25-29,30-34,...,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+
0,E02004297,E06000047,female,206,155,177,224,185,240,262,...,340,321,260,270,241,179,146,111,73,36
1,E02004290,E06000047,female,159,172,162,170,141,158,182,...,290,216,230,220,185,143,92,42,30,15
2,E02004298,E06000047,female,255,222,275,262,180,226,247,...,411,397,349,385,272,248,215,137,106,68
3,E02004299,E06000047,female,235,194,201,247,244,260,262,...,307,314,298,277,233,193,149,133,113,55
4,E02004291,E06000047,female,195,178,191,199,176,220,210,...,289,251,202,222,185,178,133,111,57,38


<br>

## Set-up


### Populations & Aggregates

<br>

**MSOA**

In [11]:
edges = populations.drop(columns='sex').groupby(by=['msoa', 'ltla']).agg('sum').sum(axis=1)
logger.info(edges.head(3))
logger.info(type(edges))

edges.rename('ppln_msoa', inplace=True)
edges = edges.to_frame().reset_index(drop=False)
logger.info(edges.head())

logger.info(edges.shape)


msoa       ltla     
E02000001  E09000001     7412
E02000002  E09000002     6783
E02000003  E09000002    10088
dtype: int64
2021-12-19 21:50:15.473


<class 'pandas.core.series.Series'>
2021-12-19 21:50:15.475


        msoa       ltla  ppln_msoa
0  E02000001  E09000001       7412
1  E02000002  E09000002       6783
2  E02000003  E09000002      10088
3  E02000004  E09000002       6185
4  E02000005  E09000002       8588
2021-12-19 21:50:15.479


(6791, 3)
2021-12-19 21:50:15.484



<br>
<br>

**LTLA**

In [12]:
nodes = edges.copy().drop(columns='msoa').groupby(by='ltla').agg(ppln_ltla=('ppln_msoa', sum))
logger.info(nodes.head(3))
logger.info(type(nodes))

nodes.reset_index(drop=False, inplace=True)
logger.info(nodes.head())

logger.info(nodes.shape)


           ppln_ltla
ltla                
E06000001      92088
E06000002     138368
E06000003     135164
2021-12-19 21:50:15.514


<class 'pandas.core.frame.DataFrame'>
2021-12-19 21:50:15.516


        ltla  ppln_ltla
0  E06000001      92088
1  E06000002     138368
2  E06000003     135164
3  E06000004     191824
4  E06000005     105584
2021-12-19 21:50:15.520


(314, 2)
2021-12-19 21:50:15.523



<br>
<br>

**MSOA & LTLA**

In [13]:
aggregates = edges.merge(nodes, how='left', on='ltla')
logger.info(aggregates.head())
logger.info(aggregates.shape)


        msoa       ltla  ppln_msoa  ppln_ltla
0  E02000001  E09000001       7412       7412
1  E02000002  E09000002       6783     187029
2  E02000003  E09000002      10088     187029
3  E02000004  E09000002       6185     187029
4  E02000005  E09000002       8588     187029
2021-12-19 21:50:15.537


(6791, 4)
2021-12-19 21:50:15.541



<br>

Simple checks

In [14]:
check = aggregates.groupby(by=['ltla', 'ppln_ltla']).agg(n=('ppln_msoa', sum))
check.reset_index(drop=False, inplace=True)
assert (check.ppln_ltla - check.n).sum() == 0, 'the population of a LTLA determined via its child MSOA codes must equal the LTLA population'

In [15]:
assert aggregates.loc[aggregates.ltla == 'E09000002', 'ppln_msoa'].sum() == 187029, 'The sum of the MSOA populations of a LTLA must equal the LTLA population'

<br>
<br>

**`populations`: Append Aggregates**

In [16]:
populations = populations.merge(aggregates, how='left', on=['msoa', 'ltla'])
logger.info(populations.shape)

populations.head()


(13582, 24)
2021-12-19 21:50:15.613



,msoa,ltla,sex,0-4,5-9,10-14,15-19,20-24,25-29,30-34,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+,ppln_msoa,ppln_ltla
0,E02004297,E06000047,female,206,155,177,224,185,240,262,...,260,270,241,179,146,111,73,36,7900,512994
1,E02004290,E06000047,female,159,172,162,170,141,158,182,...,230,220,185,143,92,42,30,15,5990,512994
2,E02004298,E06000047,female,255,222,275,262,180,226,247,...,349,385,272,248,215,137,106,68,9706,512994
3,E02004299,E06000047,female,235,194,201,247,244,260,262,...,298,277,233,193,149,133,113,55,8455,512994
4,E02004291,E06000047,female,195,178,191,199,176,220,210,...,202,222,185,178,133,111,57,38,6788,512994


<br>
<br>

### Patients & Populations

In [17]:
merged = populations.merge(patients.drop(columns='catchment_year'), how='right', on='msoa')
merged.head()

,msoa,ltla,sex,0-4,5-9,10-14,15-19,20-24,25-29,30-34,...,70-74,75-79,80-84,85-89,90+,ppln_msoa,ppln_ltla,trust_code,patients_from_msoa_to_trust,total_patients_of_msoa
0,E02001214,E08000007,female,181,198,230,234,158,160,155,...,190,199,168,107,53,7445,283253,R0A,1981,3418
1,E02001214,E08000007,male,224,216,215,251,221,136,136,...,200,148,123,65,25,7445,283253,R0A,1981,3418
2,E02001214,E08000007,female,181,198,230,234,158,160,155,...,190,199,168,107,53,7445,283253,RBV,90,3418
3,E02001214,E08000007,male,224,216,215,251,221,136,136,...,200,148,123,65,25,7445,283253,RBV,90,3418
4,E02001214,E08000007,female,181,198,230,234,158,160,155,...,190,199,168,107,53,7445,283253,RJN,27,3418


<br>
<br>

### Patients & Aggregates

<br>

**Patients from LTLA to trust**

In [18]:
reference = merged.copy()[['trust_code', 'msoa', 'ltla', 'ppln_ltla', 'patients_from_msoa_to_trust']]
reference.drop_duplicates(inplace=True)
logger.info(reference.head())

logger.info(reference.ltla.unique().shape)

values = reference.groupby(by=['trust_code', 'ltla']).agg(patients_from_ltla_to_trust=('patients_from_msoa_to_trust', sum))
values.reset_index(drop=False, inplace=True)
values.info()


  trust_code       msoa       ltla  ppln_ltla  patients_from_msoa_to_trust
0        R0A  E02001214  E08000007     283253                         1981
2        RBV  E02001214  E08000007     283253                           90
4        RJN  E02001214  E08000007     283253                           27
6        RM3  E02001214  E08000007     283253                           68
8        RRF  E02001214  E08000007     283253                           13
2021-12-19 21:50:15.840


(314,)
2021-12-19 21:50:15.850



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4345 entries, 0 to 4344
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   trust_code                   4345 non-null   object
 1   ltla                         4345 non-null   object
 2   patients_from_ltla_to_trust  4345 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 102.0+ KB


<br>

Hence

In [19]:
extended = merged.copy().merge(values, how='left', on=['trust_code', 'ltla'])
extended.head()

,msoa,ltla,sex,0-4,5-9,10-14,15-19,20-24,25-29,30-34,...,75-79,80-84,85-89,90+,ppln_msoa,ppln_ltla,trust_code,patients_from_msoa_to_trust,total_patients_of_msoa,patients_from_ltla_to_trust
0,E02001214,E08000007,female,181,198,230,234,158,160,155,...,199,168,107,53,7445,283253,R0A,1981,3418,29065
1,E02001214,E08000007,male,224,216,215,251,221,136,136,...,148,123,65,25,7445,283253,R0A,1981,3418,29065
2,E02001214,E08000007,female,181,198,230,234,158,160,155,...,199,168,107,53,7445,283253,RBV,90,3418,2340
3,E02001214,E08000007,male,224,216,215,251,221,136,136,...,148,123,65,25,7445,283253,RBV,90,3418,2340
4,E02001214,E08000007,female,181,198,230,234,158,160,155,...,199,168,107,53,7445,283253,RJN,27,3418,1281


<br>
<br>

**All LTLA Patients**

In [20]:
reference = merged.copy()[[ 'msoa', 'ltla', 'total_patients_of_msoa']]
reference.drop_duplicates(inplace=True)
logger.info(reference.head())

logger.info(reference.ltla.unique().shape)

values = reference.groupby(by='ltla').agg(total_patients_of_ltla=('total_patients_of_msoa', sum))
values.reset_index(drop=False, inplace=True)
values.info()


         msoa       ltla  total_patients_of_msoa
0   E02001214  E08000007                    3418
14  E02001240  E08000008                    2944
26  E02001057  E08000003                    3421
40  E02006902  E08000003                    1791
50  E02001056  E08000003                    4350
2021-12-19 21:50:16.081


(314,)
2021-12-19 21:50:16.085



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ltla                    314 non-null    object
 1   total_patients_of_ltla  314 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.0+ KB


<br>

Hence

In [21]:
extended = extended.copy().merge(values, how='left', on='ltla')
extended.head()

,msoa,ltla,sex,0-4,5-9,10-14,15-19,20-24,25-29,30-34,...,80-84,85-89,90+,ppln_msoa,ppln_ltla,trust_code,patients_from_msoa_to_trust,total_patients_of_msoa,patients_from_ltla_to_trust,total_patients_of_ltla
0,E02001214,E08000007,female,181,198,230,234,158,160,155,...,168,107,53,7445,283253,R0A,1981,3418,29065,124546
1,E02001214,E08000007,male,224,216,215,251,221,136,136,...,123,65,25,7445,283253,R0A,1981,3418,29065,124546
2,E02001214,E08000007,female,181,198,230,234,158,160,155,...,168,107,53,7445,283253,RBV,90,3418,2340,124546
3,E02001214,E08000007,male,224,216,215,251,221,136,136,...,123,65,25,7445,283253,RBV,90,3418,2340,124546
4,E02001214,E08000007,female,181,198,230,234,158,160,155,...,168,107,53,7445,283253,RJN,27,3418,1281,124546


<br>
<br>

## Case: MSOA

In [22]:
msoa_ = extended.copy()

# trust fraction of patients (tfp) w.r.t. MSOA
msoa_.loc[:, 'tfp_msoa'] = np.true_divide(msoa_.patients_from_msoa_to_trust, msoa_.total_patients_of_msoa)

# estimated trust catchment (etc) w.r.t. MSOA
msoa_.loc[:, 'etc_msoa'] = np.multiply(msoa_.tfp_msoa, msoa_.ppln_msoa)

msoa_ = msoa_[['trust_code', 'msoa', 'ltla', 'ppln_msoa', 'ppln_ltla', 'sex'] + age_groups + ['patients_from_msoa_to_trust', 'total_patients_of_msoa', 'tfp_msoa', 'etc_msoa']]

msoa_.head()

,trust_code,msoa,ltla,ppln_msoa,ppln_ltla,sex,0-4,5-9,10-14,15-19,...,65-69,70-74,75-79,80-84,85-89,90+,patients_from_msoa_to_trust,total_patients_of_msoa,tfp_msoa,etc_msoa
0,R0A,E02001214,E08000007,7445,283253,female,181,198,230,234,...,245,190,199,168,107,53,1981,3418,0.579579,4314.963429
1,R0A,E02001214,E08000007,7445,283253,male,224,216,215,251,...,198,200,148,123,65,25,1981,3418,0.579579,4314.963429
2,RBV,E02001214,E08000007,7445,283253,female,181,198,230,234,...,245,190,199,168,107,53,90,3418,0.026331,196.035693
3,RBV,E02001214,E08000007,7445,283253,male,224,216,215,251,...,198,200,148,123,65,25,90,3418,0.026331,196.035693
4,RJN,E02001214,E08000007,7445,283253,female,181,198,230,234,...,245,190,199,168,107,53,27,3418,0.007899,58.810708


In [23]:
for trust in msoa_.trust_code.unique()[:1]:
    
    logger.info(trust)
    
    sample = msoa_.copy().loc[msoa_.trust_code == trust, :]
    sample.drop(columns='trust_code', inplace=True)
    
    leaf = sample.copy().melt(id_vars=['msoa', 'ltla', 'ppln_msoa', 'ppln_ltla',  'patients_from_msoa_to_trust', 'total_patients_of_msoa', 
                                       'tfp_msoa', 'etc_msoa', 'sex'], 
                              var_name='ag', value_name='ag_ppln_msoa')
    
    # age group fraction of MSOA population
    leaf.loc[:, 'agf_ppln_msoa'] = np.true_divide(leaf.ag_ppln_msoa, leaf.ppln_msoa)
    
    # age group [nhs] trust factor
    leaf.loc[:, 'ag_trust_factor'] = np.multiply(leaf.tfp_msoa, leaf.agf_ppln_msoa)



R0A
2021-12-19 21:50:16.328



In [24]:
leaf.head()

,msoa,ltla,ppln_msoa,ppln_ltla,patients_from_msoa_to_trust,total_patients_of_msoa,tfp_msoa,etc_msoa,sex,ag,ag_ppln_msoa,agf_ppln_msoa,ag_trust_factor
0,E02001214,E08000007,7445,283253,1981,3418,0.579579,4314.963429,female,0-4,181,0.024312,0.014090
1,E02001214,E08000007,7445,283253,1981,3418,0.579579,4314.963429,male,0-4,224,0.030087,0.017438
2,E02001240,E08000008,6190,219727,1181,2944,0.401155,2483.148777,female,0-4,177,0.028595,0.011471
3,E02001240,E08000008,6190,219727,1181,2944,0.401155,2483.148777,male,0-4,197,0.031826,0.012767
4,E02001057,E08000003,8119,502902,1752,3421,0.512131,4157.991231,female,0-4,221,0.027220,0.013940


<br>
<br>

### Tests

In [25]:
test = leaf.loc[leaf.msoa == 'E02001214', :]
logger.info('patients from msoa to trust: %s', (test.total_patients_of_msoa * test.ag_trust_factor).sum())
logger.info('estimated trust catchment w.r.t. msoa: %s', (test.ppln_msoa * test.ag_trust_factor).sum())


patients from msoa to trust: 1981.0
2021-12-19 21:50:16.416


estimated trust catchment w.r.t. msoa: 4314.963428905792
2021-12-19 21:50:16.419



<br>
<br>

## Case: LTLA

<br>

**Baseline**

In [26]:
baseline = extended[['trust_code', 'ltla', 'ppln_ltla', 'patients_from_ltla_to_trust', 'total_patients_of_ltla']].drop_duplicates()
baseline.head()

,trust_code,ltla,ppln_ltla,patients_from_ltla_to_trust,total_patients_of_ltla
0,R0A,E08000007,283253,29065,124546
2,RBV,E08000007,283253,2340,124546
4,RJN,E08000007,283253,1281,124546
6,RM3,E08000007,283253,2520,124546
8,RRF,E08000007,283253,107,124546


<br>

**Age Groups Aggregates**

In [27]:
reference = extended.copy()[['msoa', 'ltla', 'sex'] + age_groups]
reference.drop_duplicates(inplace=True)

values = reference.copy().drop(columns='msoa').groupby(by=['ltla', 'sex']).agg('sum')
values.reset_index(drop=False, inplace=True)
values.head()

,ltla,sex,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,...,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+
0,E06000001,female,2811,2549,2780,3104,2930,2977,2672,2648,...,3677,3274,2848,2929,2186,2125,1848,1311,862,429
1,E06000001,male,2954,2619,2839,3177,2959,2703,2400,2628,...,3537,3251,2753,2898,2136,1775,1470,935,475,136
2,E06000002,female,4707,4119,4198,4870,5534,4970,4173,4119,...,5075,4691,4100,3797,2928,2735,2417,1864,1105,624
3,E06000002,male,4772,4247,4211,5057,6153,4838,4108,3944,...,4801,4535,3851,3744,2830,2385,1868,1278,590,211
4,E06000003,female,3660,3389,3793,4268,4020,4001,3649,3897,...,5366,4791,4389,4847,4048,3483,2705,2168,1377,743


<br>

**Baseline & Age Goups Aggregates**

In [28]:
ltla_ = baseline.merge(values, how='left', on='ltla')
ltla_.head()

,trust_code,ltla,ppln_ltla,patients_from_ltla_to_trust,total_patients_of_ltla,sex,0-4,5-9,10-14,15-19,...,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+
0,R0A,E08000007,283253,29065,124546,female,8424,7654,8210,8211,...,11318,9823,8504,9116,7580,6455,5596,4548,2891,1728
1,R0A,E08000007,283253,29065,124546,male,8918,8269,8467,8819,...,10882,9856,8760,9023,6970,5678,4589,3146,1608,606
2,RBV,E08000007,283253,2340,124546,female,8424,7654,8210,8211,...,11318,9823,8504,9116,7580,6455,5596,4548,2891,1728
3,RBV,E08000007,283253,2340,124546,male,8918,8269,8467,8819,...,10882,9856,8760,9023,6970,5678,4589,3146,1608,606
4,RJN,E08000007,283253,1281,124546,female,8424,7654,8210,8211,...,11318,9823,8504,9116,7580,6455,5596,4548,2891,1728


<br>

**Additional Features**

In [29]:
# trust fraction of patients w.r.t. LTLA
ltla_.loc[:, 'tfp_ltla'] = np.true_divide(ltla_.patients_from_ltla_to_trust, ltla_.total_patients_of_ltla)

# estimated [nhs] trust catchment w.r.t. LTLA
ltla_.loc[:, 'etc_ltla'] = np.multiply(ltla_.tfp_ltla, ltla_.ppln_ltla)

ltla_.head()

,trust_code,ltla,ppln_ltla,patients_from_ltla_to_trust,total_patients_of_ltla,sex,0-4,5-9,10-14,15-19,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+,tfp_ltla,etc_ltla
0,R0A,E08000007,283253,29065,124546,female,8424,7654,8210,8211,...,8504,9116,7580,6455,5596,4548,2891,1728,0.233368,66102.070279
1,R0A,E08000007,283253,29065,124546,male,8918,8269,8467,8819,...,8760,9023,6970,5678,4589,3146,1608,606,0.233368,66102.070279
2,RBV,E08000007,283253,2340,124546,female,8424,7654,8210,8211,...,8504,9116,7580,6455,5596,4548,2891,1728,0.018788,5321.825029
3,RBV,E08000007,283253,2340,124546,male,8918,8269,8467,8819,...,8760,9023,6970,5678,4589,3146,1608,606,0.018788,5321.825029
4,RJN,E08000007,283253,1281,124546,female,8424,7654,8210,8211,...,8504,9116,7580,6455,5596,4548,2891,1728,0.010285,2913.358060


<br>

**Distributing**

In [30]:
for trust in ltla_.trust_code.unique()[:1]:
    
    logger.info(trust)
    
    sample = ltla_.copy().loc[ltla_.trust_code == trust, :]
    sample.drop(columns='trust_code', inplace=True)
    
    stem = sample.copy().melt(id_vars=['ltla', 'ppln_ltla',  'patients_from_ltla_to_trust', 'total_patients_of_ltla', 
                                       'tfp_ltla', 'etc_ltla', 'sex'], var_name='ag', value_name='ag_ppln_ltla')
    
    # age group fraction of LTLA population
    stem.loc[:, 'agf_ppln_ltla'] = np.true_divide(stem.ag_ppln_ltla, stem.ppln_ltla)
    
    # age group [nhs] trust factor
    stem.loc[:, 'ag_trust_factor'] = np.multiply(stem.tfp_ltla, stem.agf_ppln_ltla)



R0A
2021-12-19 21:50:16.760



In [31]:
stem

,ltla,ppln_ltla,patients_from_ltla_to_trust,total_patients_of_ltla,tfp_ltla,etc_ltla,sex,ag,ag_ppln_ltla,agf_ppln_ltla,ag_trust_factor
0,E08000007,283253,29065,124546,0.233368,66102.070279,female,0-4,8424,0.029740,6.940398e-03
1,E08000007,283253,29065,124546,0.233368,66102.070279,male,0-4,8918,0.031484,7.347397e-03
2,E08000008,219727,17901,101971,0.175550,38573.055349,female,0-4,7126,0.032431,5.693286e-03
3,E08000008,219727,17901,101971,0.175550,38573.055349,male,0-4,7391,0.033637,5.905007e-03
4,E08000003,502902,140848,202651,0.695027,349530.675378,female,0-4,17834,0.035462,2.464719e-02
...,...,...,...,...,...,...,...,...,...,...,...
1895,E07000163,55459,38,22684,0.001675,92.904338,male,90+,166,0.002993,5.014181e-06
1896,E07000030,52502,31,19821,0.001564,82.113011,female,90+,357,0.006800,1.063478e-05
1897,E07000030,52502,31,19821,0.001564,82.113011,male,90+,125,0.002381,3.723662e-06
1898,E08000021,279092,8,109985,0.000073,20.300368,female,90+,1462,0.005238,3.810277e-07


<br>
<br>

### Tests

In [32]:
test = stem.loc[stem.ltla == 'E08000007', :]
logger.info('patients from ltla to trust: %s', (test.total_patients_of_ltla * test.ag_trust_factor).sum())
logger.info('estimated trust catchment w.r.t. ltla: %s', (test.ppln_ltla * test.ag_trust_factor).sum())


patients from ltla to trust: 29064.999999999996
2021-12-19 21:50:16.828


estimated trust catchment w.r.t. ltla: 66102.07027925426
2021-12-19 21:50:16.830

